In [1]:
from jacobi import Planner, Frame, CartesianWaypoint, LinearMotion
from jacobi.robots import ABBYuMiIRB14000 as Yumi
from jacobi.drivers import ABBDriver

yumi = Yumi()
yumi.set_speed(0.14)

planner = Planner(yumi)

# From a fresh robot start, need to run twice to upload T_ROB_R before T_ROB_L starts

module = ABBDriver.RapidModule(unit='ROB_L')
module.upload = False # TURN THIS TO FALSE OTHERWISE EDITED PENDANT MODULES GET OVERWRITTEN
driver_left = ABBDriver(
    planner, yumi.left,
    host='192.168.125.1', port=6511,
    module=module, version=ABBDriver.RobotWareVersion.RobotWare6,
)

module = ABBDriver.RapidModule(unit='ROB_R')
module.upload = False # TURN THIS TO FALSE OTHERWISE EDITED PENDANT MODULES GET OVERWRITTEN
driver_right = ABBDriver(
    planner, yumi.right,
    host='192.168.125.1', port=6512,
    module=module, version=ABBDriver.RobotWareVersion.RobotWare6,
)


l_g_comm = ['192.168.125.40', 'Hand_L']
r_g_comm = ['192.168.125.30', 'Hand_R']

def get_signal(driver, comm, signal):
    return driver.rws.get_signal(signal = signal, network = comm[0], device = comm[1])

def set_signal(driver, comm, signal, value):
    return driver.rws.set_signal(signal = signal, value = str(value), network = comm[0], device = comm[1])


In [2]:
import time

print('Left current position', driver_left.current_joint_position)
print('Right current position', driver_right.current_joint_position)

# Move both arms one after the other
home_left = [-0.967, -0.734, 0.748, -0.118, 1.365, 1.477, -1.6]
home_right = [0.740, -0.319, -0.876, -0.310, -0.866, 1.276, -1.8]

driver_left.move_to(home_left)
driver_right.move_to(home_right)

# Calculate tcp position in Task space
print(yumi.left.calculate_tcp(home_left))
print(yumi.right.calculate_tcp(home_right))

# Move both arms synchronized
trajectory = planner.plan(
    start={
        yumi.left: driver_left.current_joint_position,
        yumi.right: driver_right.current_joint_position,
    },
    goal={
        yumi.left: CartesianWaypoint(Frame(x=0.60, y=0.24, z=0.25, b=-3.1415, c=1.85)),
        yumi.right: CartesianWaypoint(Frame(x=0.50, y=-0.14, z=0.25, b=-3.1415, c=1.12)),
    },
)

result_left = driver_left.run_async(trajectory)
result_right = driver_right.run_async(trajectory)
await result_left
await result_right

# Linear motion of left arm
motion = LinearMotion(
    yumi.left,
    driver_left.current_joint_position,
    Frame(x=0.60, y=0.14, z=0.25, b=-3.1415, c=1.0),
)
motion.robot.set_speed(0.05)

tl = planner.plan(motion)
result_left = driver_left.run_async(tl)

# And stop after a timeout because we can control the robot on-the-fly
set_signal(driver_left, l_g_comm, 'cmd_GripperState_L', 5)
set_signal(driver_left, l_g_comm, 'cmd_GripperPos_L', 0)
print(get_signal(driver_left, l_g_comm, 'cmd_GripperState_L'))
print(get_signal(driver_left, l_g_comm, 'cmd_GripperPos_L'))
driver_left.rws.call_procedure('handleRunSGRoutine')
time.sleep(3.0)
driver_left.stop()

# Back to home
yumi.set_speed(0.15)
result_left = driver_left.move_to_async(home_left)
result_right = driver_right.move_to_async(home_right)
await result_left
await result_right


Left current position [-0.9671500527847318, -0.7343817840285362, 0.7489273706913036, -0.11783803724095614, 1.3649358387445345, 1.4782524110210244, -1.5989094542101]
Right current position [0.7399978581352953, -0.3190054195945965, -0.8768290109937217, -0.3091502921122174, -0.8658609152022848, 1.2794772630508184, -1.798314173462001]
<jacobi.Frame with position=[0.5789, 0.2744, 0.4075], orientation=[0.0163, -3.0607, 1.8566]>
<jacobi.Frame with position=[0.6075, -0.2171, 0.4156], orientation=[0.0206, -3.0378, 1.1277]>
Signal(cmd_GripperState_L, 5)
Signal(cmd_GripperPos_L, 0)
[jacobi.driver] Position deviation of current and desired state exceed safety threshold of 0.100000 [rad].


[1] Finished successfully.

In [3]:
print(get_signal(driver_left, l_g_comm, 'cmd_GripperState_L'))
print(get_signal(driver_left, l_g_comm, 'cmd_GripperPos_L'))
set_signal(driver_left, l_g_comm, 'cmd_GripperState_L', 4)
set_signal(driver_left, l_g_comm, 'cmd_GripperPos_L', 0)
print(get_signal(driver_left, l_g_comm, 'cmd_GripperState_L'))
print(get_signal(driver_left, l_g_comm, 'cmd_GripperPos_L'))
driver_left.rws.call_procedure('handleRunSGRoutine')

Signal(cmd_GripperState_L, 5)
Signal(cmd_GripperPos_L, 0)
Signal(cmd_GripperState_L, 4)
Signal(cmd_GripperPos_L, 0)


In [10]:

driver_left.rws.call_procedure('handleRunSGRoutine')